In [1]:
import pandas as pd
import numpy as np
import os
import requests
from io import BytesIO
import json
import yadisk
from datetime import datetime, date, timedelta
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import config
from yandex_disk_func import *
from parse_functions import *
from google_connector import *
from date_time_func import *

# забираем токен для подключения к гугл
service_key = config.service
google_sheet_link = 'https://docs.google.com/spreadsheets/d/1xIVvTrbMUOhGiz6aA4jx9CRspigGgljVzqAJzsLLQtQ/edit?usp=sharing' 
gmail = config.gmail
sheet_name = 'bench_raw'

# обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета
public_key = config.public_key 

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# создаем словарь, где сохраним каждый продукт в отдельный датаФрейм со своим ключом
main_dict = {}

In [4]:
get_data_from_ya_folder(yandex_folders, main_dict, flag='split', extention='pdf')

/01_yandex
/02_prog
/03_split


Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)
Ignoring wrong pointing object 125 0 (offset 0)
Ignoring wrong pointing object 140 0 (offset 0)


бенчмарки сентябрь 2024
26
бенчмарки vk реклама
бенчмарки vk реклама в категории авто
бенчмарки vk рекламав категории ecom
бенчмарки vk реклама в категориифинансы
бенчмарки vk реклама в категорииfmcg
бенчмарки vk реклама в категориимедицина и здоровье
бенчмарки vk реклама в категориинедвижимость
бенчмаркиmytarget
бенчмаркиmytarget в категорииавто 
бенчмаркиmytarget в категории ecom
бенчмаркиmytarget в категории финансы
бенчмаркиmytarget в категории fmcg
бенчмаркиmytarget в категории медицина и здоровье
бенчмаркиmytarget в категории недвижимость
бенчмаркивконтакте
бенчмаркивконтактев категории авто
бенчмаркивконтактев категории ecom
бенчмаркивконтактев категории финансы
бенчмаркивконтактев категории fmcg
бенчмаркивконтактев категории медицина и здоровье
бенчмаркивконтактев категории недвижимость


In [5]:
df = pd.concat(main_dict, ignore_index=True)
df = df.fillna('')
df = df[['период', 'source', 'категория', 'объект рекламы', 'цель рекламы', 'format_type', 'cpc без ндс', 'cpm без ндс', 'ctr %']]

In [6]:
df.head()

,период,source,категория,объект рекламы,цель рекламы,format_type,cpc без ндс,cpm без ндс,ctr %
0,01.09.2024-30.09.2024,VK Ads,авто,Mini app,Переходы в VK Mini Apps,,22.0,140.9,0.64
1,01.09.2024-30.09.2024,VK Ads,авто,Видео и трансляции,"Показы рекламы (запись, видео или клип)",,47.7,70.6,0.15
2,01.09.2024-30.09.2024,VK Ads,авто,Видео и трансляции,Продвижение трансляций,,47.2,71.5,0.15
3,01.09.2024-30.09.2024,VK Ads,авто,Дзен,Клики по рекламе,,5.8,97.9,1.68
4,01.09.2024-30.09.2024,VK Ads,авто,Дзен,Подписчики,,13.2,67.4,0.51


In [7]:
float_columns = ['cpc без ндс', 'cpm без ндс', 'ctr %']
df[float_columns] = df[float_columns].apply(lambda x: x.astype('float')).round(2)

In [8]:
df.head()

,период,source,категория,объект рекламы,цель рекламы,format_type,cpc без ндс,cpm без ндс,ctr %
0,01.09.2024-30.09.2024,VK Ads,авто,Mini app,Переходы в VK Mini Apps,,22.0,140.9,0.64
1,01.09.2024-30.09.2024,VK Ads,авто,Видео и трансляции,"Показы рекламы (запись, видео или клип)",,47.7,70.6,0.15
2,01.09.2024-30.09.2024,VK Ads,авто,Видео и трансляции,Продвижение трансляций,,47.2,71.5,0.15
3,01.09.2024-30.09.2024,VK Ads,авто,Дзен,Клики по рекламе,,5.8,97.9,1.68
4,01.09.2024-30.09.2024,VK Ads,авто,Дзен,Подписчики,,13.2,67.4,0.51


In [9]:
# создаем подключение к Гуглу
client = create_connection(service_key)

Connection established successfully...


In [10]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [11]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, df)

DataFrame exported successfully...
